<a href="https://colab.research.google.com/github/pollinations/hive/blob/main/notebooks/6%20Image-To-Image/1%20ruRealESRGAN%20Super-Resolution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://ipfs.pollinations.ai/ipfs/QmbPCzFiLrskHNQKs5eM8CGkEQ6qvirBTmvGLkKVWRcCwX" width="300" height="300" />

Scale up images or videos, turning them HD.

---
Uses [Real-ESRGAN](https://github.com/xinntao/Real-ESRGAN) to perform neural super-resolution (creates HD images or videos from low-resolution).

The modelds were fine-tuned by russian company [Sber](https://github.com/sberbank-ai/ru-dalle).

In [ ]:

# The image or video to scale up
image_file = ''  #@param {type: "string", accepted_files: "video,image"}

# By what amount to upscale the image
upscale_factor = "x4"  #@param ['x2', 'x4', 'x8']

output_path = "/content"


In [ ]:
video_file = image_file

import mimetypes
mimetypes.init()

mimestart = mimetypes.guess_type(video_file)[0]
if mimestart is None:
    raise Exception("Could not determine file type of " + video_file)
mimestart = mimestart.split('/')[0]
is_video = mimestart == 'video'

input_path = "/tmp/sr_input"
!rm -rf $input_path
!mkdir -p $input_path

# Copy image or extract video frames and copy to input_path

if is_video:
    # Save video framerate
    framerate = !ffprobe -v error -select_streams v:0 -show_entries stream=r_frame_rate -of default=noprint_wrappers=1:nokey=1 "$video_file"
    multiplier, divisor = framerate[0].split('/')
    framerate = float(multiplier) / float(divisor)
    print("framerate", framerate)

    # Extract frames
    !ffmpeg -i "$video_file" $input_path/%05d.jpg
else:
    !cp "$video_file" $input_path/





Веса для x2, x4, x8 моделей можно скачать по этой ссылке:
https://drive.google.com/drive/folders/16PlVKhTNkSyWFx52RPb2hXPIQveNGbxS

In [ ]:

model_file = f"RealESRGAN_{upscale_factor}.pth"
model_url = f"https://nftstorage.link/ipfs/QmYKeprXZFBxhmJAgmtUhaoaVQv6fSibqGwtPMj7AEoWbk/{model_file}"

!git clone https://github.com/sberbank-ai/Real-ESRGAN
%cd /content/Real-ESRGAN
!pip install -r requirements.txt

!wget -N {model_url}

Cloning into 'Real-ESRGAN'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 20 (delta 2), reused 17 (delta 2), pack-reused 0
Unpacking objects: 100% (20/20), done.
/content/Real-ESRGAN
--2021-11-04 05:13:26--  https://ipfs.pollinations.ai/ipfs/QmbgtAqZyBTUhyUFh6ZZD5HbgWJBSNnnPrzj72k4ikq7PV/SBER_RealESRGAN_x4.pth
Resolving pollinations.ai (pollinations.ai)... 50.18.120.10, 138.197.207.178, 2604:a880:2:d0::212a:f001, ...
Connecting to pollinations.ai (pollinations.ai)|50.18.120.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 67040989 (64M) [application/zip]
Saving to: ‘SBER_RealESRGAN_x4.pth’

SBER_RealESRGAN_x4. 100%[===================>]  63.93M  20.8MB/s    in 3.5s    

2021-11-04 05:13:30 (18.2 MB/s) - ‘SBER_RealESRGAN_x4.pth’ saved [67040989/67040989]



# Загрузка изображений/архивов


# Запуск модели

In [ ]:
# Создание модели
from realesrgan import RealESRGAN
from PIL import Image
import numpy as np
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('device:', device)

scale_map = {
    "x2":2, "x4":4, "x8":8
}

model = RealESRGAN(device, scale=scale_map[upscale_factor])
model.load_weights(model_file)

device: cuda


Улучшение одного изображения:

In [ ]:
from glob import glob
from tqdm import tqdm

images = glob(f"{input_path}/*.jpg") + glob(f"{input_path}/*.png") + glob(f"{input_path}/*.jpeg")
images = list(sorted(images))

tmp_output = "/tmp/sr_output"
!rm -rf $tmp_output
!mkdir -p $tmp_output

for image_file in tqdm(images):

    #print("image_file", image_file)
    
    image = Image.open(image_file).convert('RGB')
    sr_image = model.predict(np.array(image))

    # Get the filename and extension
    filename = image_file.split('/')[-1]
    ext = filename.split('.')[-1]
   
    # Save to tmp_output
    out_file = f"{tmp_output}/{filename}"
    sr_image.save(out_file)

    !cp "$out_file" $output_path/frame.{ext}

if is_video:
    # Create video
    !ffmpeg -framerate $framerate -i {tmp_output}/%05d.jpg -c:v libx264 -pix_fmt yuv420p $input_path/output.mp4
    
    # Check if original video contains audio stream
    has_audio_stream = !ffprobe -v error -select_streams a -show_entries stream=codec_name -of default=noprint_wrappers=1:nokey=1 "$video_file"
    print("has_audio_stream",has_audio_stream)
    has_audio_stream = len(has_audio_stream) > 0

    if has_audio_stream:
        # Combine with audio of original video
        !ffmpeg -i $input_path/output.mp4 -i "$video_file" -c copy -map 0:v -map 1:a -shortest $output_path/output.mp4
    
    else:
        # Copy output video
        !cp $input_path/output.mp4 $output_path/output.mp4
        
    import os.path
    if not os.path.exists(output_path + '/output.mp4'):
        raise Exception("Expected output file does not exist.")
    !rm {output_path}/*.jpg
